## Get predictions using Kipoi embedded model

We need (at least):
- dl_kwargs = gtf_file, fasta_file, vcf_file
- output file = predictions.tsv'

In [14]:
### Standalone script
import kipoi
from kipoi_cadd.writers import LmdbWriter
from kipoi_cadd.utils import load_pickle
from kipoi_cadd.data_utils import calculate_map_size
import kipoi_veff.snv_predict as sp
from kipoi_veff.scores import Logit
from kipoi import get_model
import os

training_dir_hg37 = "/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37"
gtf_file = "/s/genomes/human/hg38/Homo_sapiens.GRCh38.79.chr-prepend.gtf"
# gtf_file = "/s/genomes/human/hg19/gencode28/gencode.v28.annotation.gff3.gz"
fasta_file = "/s/genomes/human/hg38/hg38.fa"
vcf_file = os.path.join(training_dir_hg37, "example_10.vcf")
lmdb_mmsplice = os.path.join(training_dir_hg37, "lmdb/MMSplice_deltaLogitPSI")
if not os.path.exists(lmdb_mmsplice):
    os.makedirs(lmdb_mmsplice)

model = get_model("MMSplice/deltaLogitPSI")
dl_kwargs = {'gtf_file': gtf_file, 'fasta_file': fasta_file, 'vcf_file': vcf_file}
dataloader = model.default_dataloader
# dl = model.default_dataloader(**dl_kwargs)

In [ ]:
%%time
pred = model.pipeline.predict(dl_kwargs, batch_size=4, num_workers=64)

## Get predictions from their package

In [1]:
%%time # 22mins
# Import
from mmsplice.vcf_dataloader import SplicingVCFDataloader
from mmsplice import MMSplice, predict_all_table
from mmsplice.utils import max_varEff

# example files
gtf = "/s/genomes/human/hg38/Homo_sapiens.GRCh38.79.chr-prepend.gtf"
vcf = "/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37/sample_chr22_prefixed.vcf"
fasta = "/s/genomes/human/hg38/hg38.fa"

# dataloader to load variants from vcf
dl = SplicingVCFDataloader(gtf,
                          fasta,
                          vcf,
                          split_seq=False)

# Specify model
model = MMSplice(
    exon_cut_l=0,
    exon_cut_r=0,
    acceptor_intron_cut=6,
    donor_intron_cut=6,
    acceptor_intron_len=50,
    acceptor_exon_len=3,
    donor_exon_len=5,
    donor_intron_len=13)

Using TensorFlow backend.
/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd-mms/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator HuberRegressor from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd-mms/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd-mms/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd-mms/lib/python3.6/site-pa

CPU times: user 22min 23s, sys: 36.8 s, total: 22min 59s
Wall time: 22min 46s


In [24]:
import pandas as pd
from sklearn.externals import joblib
from mmsplice.utils.postproc import transform
from pkg_resources import resource_filename

LINEAR_MODEL = joblib.load(resource_filename(
    'mmsplice', 'models/linear_model.pkl'))

def predict_batch(model,
                x,
                assembly=True,
                pathogenicity=False,
                splicing_efficiency=False):
    ''' Return the prediction as a table
        exon_scale_factor: can be determined through cross validation.
        Args:
            The mmsplice model object
            dataloader: dataloader object.
            split_seq: is the input sequence from dataloader splited?
            progress: show progress bar?
            assembly_fn: function to assemble modular predictions.
            pathogenicity: whether output prediction of pathogenicity
            splicing_efficiency: whether output prediction of splicing_efficiency
    '''

    ref_pred=model.predict_on_unsplitted_batch(x['inputs'])
    alt_pred=model.predict_on_unsplitted_batch(x['inputs_mut'])
    ID=x['metadata']['variant']['STR']
    exons=x['metadata']['annotation']
    ref_pred = pd.DataFrame(ref_pred, columns=['mmsplice_ref_acceptorIntron',
                                               'mmsplice_ref_acceptor',
                                               'mmsplice_ref_exon',
                                               'mmsplice_ref_donor',
                                               'mmsplice_ref_donorIntron'])
    alt_pred = pd.DataFrame(alt_pred, columns=['mmsplice_alt_acceptorIntron',
                                               'mmsplice_alt_acceptor',
                                               'mmsplice_alt_exon',
                                               'mmsplice_alt_donor',
                                               'mmsplice_alt_donorIntron'])
    pred = pd.DataFrame({'ID': ID, 'exons': exons})
    if assembly:
        ref_pred = ref_pred.values
        alt_pred = alt_pred.values
        X = alt_pred - ref_pred
        if pathogenicity:
            X_pathogenicity = transform(X, region_only=True)
            # design matrix for logistic model to predict pathogenicity
            X_pathogenicity = np.concatenate(
                [ref_pred, alt_pred, X_pathogenicity[:, -3:]], axis=-1)
            delt_pred = LOGISTIC_MODEL.predict_proba(X_pathogenicity)[:, 1]
            pred['mmsplice_pathogenicity'] = delt_pred
        if splicing_efficiency:
            X_splicing_efficiency = transform(X, region_only=False)
            X_splicing_efficiency = X_splicing_efficiency[:, [
                1, 2, 3, 5]]  # no intronic modules
            delt_pred = EFFICIENCY_MODEL.predict(X_splicing_efficiency)
            pred['mmsplice_dse'] = delt_pred
        X = transform(X, region_only=False)
        delt_pred = LINEAR_MODEL.predict(X)
        pred['mmsplice_dlogitPsi'] = delt_pred
    else:
        pred = pd.concat([pred, ref_pred, alt_pred], axis=1)
    return pred

/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd-mms/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator HuberRegressor from version 0.19.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [2]:
example_batch = next(dl.batch_predict_iter(batch_size=20))

Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16638865, REF=C, ALT=T, ID=.)
Variant(CHROM=chr22, POS=16638873, REF=C, ALT=G, ID=.)


/opt/modules/i12g/anaconda/3-5.0.1/envs/kipoi-cadd-mms/lib/python3.6/site-packages/mmsplice/vcf_dataloader.py:166: UserWarning: Reference not match, cannot mutate, return original sequence.
  " return original sequence.", UserWarning)


In [27]:
p = predict_batch(model, example_batch, assembly=False)

In [30]:
p.drop(columns=['exons'], inplace=True)
p.head()

,ID,mmsplice_ref_acceptorIntron,mmsplice_ref_acceptor,mmsplice_ref_exon,mmsplice_ref_donor,mmsplice_ref_donorIntron,mmsplice_alt_acceptorIntron,mmsplice_alt_acceptor,mmsplice_alt_exon,mmsplice_alt_donor,mmsplice_alt_donorIntron
0,chr22:16141761:G:['A'],-2.510676,-6.364011,-4.565398,-3.550600,0.105083,-2.510676,-6.869948,-4.565398,-3.550600,0.105083
1,chr22:16637117:T:['C'],-2.392119,-7.813437,-5.022861,1.906983,0.203514,-2.392119,-7.813437,-5.022861,1.906983,0.164537
2,chr22:16637117:T:['C'],-2.510911,5.773352,-4.781666,1.906983,0.203514,-2.510911,5.773352,-4.781666,1.906983,0.164537
3,chr22:16637117:T:['C'],-2.510911,5.773352,-4.781666,1.906983,0.203514,-2.510911,5.773352,-4.781666,1.906983,0.164537
4,chr22:16637117:T:['C'],-2.510911,5.773352,-4.781666,1.906983,0.203514,-2.510911,5.773352,-4.781666,1.906983,0.164537


In [7]:
example_batch['metadata']['variant']['CHROM']

20

In [9]:
pred = model.predict_on_unsplitted_batch(example_batch['inputs'])
pred.shape

(20, 5)

In [44]:
from kipoi.writers import HDF5BatchWriter
from kipoi_cadd.utils import load_pickle

batch_size = 10
file_path = "/s/project/kipoi-cadd/data/models/MMSplice/sample_chr22_predictions.h5"
writer = HDF5BatchWriter(file_path, chunk_size=batch_size, compression='gzip')
sample_ids_file = "/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37/variant_ids/sample_chr22.pkl"
sample_ids = load_pickle(sample_ids_file)

dl = SplicingVCFDataloader(gtf,
                          fasta,
                          vcf,
                          split_seq=False)
it = dl.batch_predict_iter(batch_size=batch_size)
batch_num = 0

for batch in it:
    pred = predict_batch(model, batch, assembly=False)
    pred.ID = [i.split('chr')[1] for i in pred.ID]
    pred.drop(columns=['exons'], inplace=True)
    print(pred.ID[:10])
    print(sample_ids[:10])
    break
    writer.batch_write(batch)
    batch_num += 1

Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16638731, REF=A, ALT=G, ID=.)
0    22:16141761:G:['A']
1    22:16637117:T:['C']
2    22:16637117:T:['C']
3    22:16637117:T:['C']
4    22:16637117:T:['C']
5    22:16638731:A:['G']
6    22:16638731:A:['G']
7    22:16638731:A:['G']
8    22:16638731:A:['G']
9    22:16638731:A:['G']
Name: ID, dtype: object
0    22:16139326:A:['G']
1    22:16139870:T:['C']
2    22:16139966:A:['G']
3    22:16140322:C:['T']
4    22:16140325:A:['G']
5    22:16141058:T:['C']
6    22:16141162:A:['G']
7    22:16141372:T:['C']
8    22:16141761:G:['A']
9    22:16141980:G:['A']
dtype: object


In [50]:
pred.merge(pd.DataFrame(sample_ids[:10], columns=["ID"]), how="outer")

,ID,mmsplice_ref_acceptorIntron,mmsplice_ref_acceptor,mmsplice_ref_exon,mmsplice_ref_donor,mmsplice_ref_donorIntron,mmsplice_alt_acceptorIntron,mmsplice_alt_acceptor,mmsplice_alt_exon,mmsplice_alt_donor,mmsplice_alt_donorIntron
0,22:16141761:G:['A'],-2.510676,-6.364011,-4.565398,-3.550600,0.105083,-2.510676,-6.869948,-4.565398,-3.550600,0.105083
1,22:16637117:T:['C'],-2.392119,-7.813437,-5.022861,1.906983,0.203514,-2.392119,-7.813437,-5.022861,1.906983,0.164537
2,22:16637117:T:['C'],-2.510911,5.773352,-4.781666,1.906983,0.203514,-2.510911,5.773352,-4.781666,1.906983,0.164537
3,22:16637117:T:['C'],-2.510911,5.773352,-4.781666,1.906983,0.203514,-2.510911,5.773352,-4.781666,1.906983,0.164537
4,22:16637117:T:['C'],-2.510911,5.773352,-4.781666,1.906983,0.203514,-2.510911,5.773352,-4.781666,1.906983,0.164537
5,22:16638731:A:['G'],-2.973741,4.360018,-4.472820,-4.380601,0.304834,-2.973741,4.360018,-4.472820,-4.380601,0.304834
6,22:16638731:A:['G'],-2.973741,4.360018,-4.040112,4.876277,0.405915,-2.973741,4.360018,-4.040112,4.876277,0.405915
7,22:16638731:A:['G'],-2.973741,4.360018,-4.040112,4.876277,0.405915,-2.973741,4.360018,-4.040112,4.876277,0.405915
8,22:16638731:A:['G'],-2.973741,4.360018,-4.040112,4.876277,0.405915,-2.973741,4.360018,-4.040112,4.876277,0.405915
9,22:16638731:A:['G'],-2.973741,4.360018,-4.040112,4.876277,0.405915,-2.973741,4.360018,-4.040112,4.876277,0.405915


In [40]:
pred = model.predict_on_unsplitted_batch(inputs['inputs'])

NameError: name 'inputs' is not defined

In [12]:
pred.shape

(32, 5)

In [13]:
%%time
# Do prediction
predictions = predict_all_table(model, dl, batch_size=1024, split_seq=False, assembly=False)

0it [00:00, ?it/s]/data/nasif12/home_if12/chengju/project/mmsplice/mmsplice/vcf_dataloader.py:166: UserWarning: Reference not match, cannot mutate, return original sequence.
  " return original sequence.", UserWarning)


Variant(CHROM=chr22, POS=16639593, REF=C, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16639610, REF=C, ALT=T, ID=.)
Variant(CHROM=chr22, POS=16639611, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16639665, REF=T, ALT=A, ID=.)
Variant(CHROM=chr22, POS=16639725, REF=T, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16639786, REF=C, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16639882, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=16639915, REF=G, ALT=C, ID=.)
Variant(CHROM=chr22, POS=16640038, REF=G, ALT=T, ID=.)
Variant(CHROM=chr22, POS=16640238, REF=G, ALT=C, ID=.)
Variant(CHROM=chr22, POS=16640273, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=16640302, REF=A, ALT=T, ID=.)
Variant(CHROM=chr22, POS=16640340, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=16640351, REF=T, ALT=A, ID=.)
Variant(CHROM=chr22, POS=16640354, REF=C, ALT=A, ID=.)
Variant(CHROM=chr22, POS=16640442, REF=A, ALT=T, ID=.)
Variant(CHROM=chr22, POS=16640443, REF=C, ALT=A, ID=.)
Variant(CHROM=chr22, POS=16640467, REF=A, ALT=T, ID=.)
Variant(CH

1it [01:35, 95.37s/it]

Variant(CHROM=chr22, POS=17553213, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17553213, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17553213, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17553213, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17553481, REF=T, ALT=C, ID=.)
Variant(CHROM=chr22, POS=17553481, REF=T, ALT=C, ID=.)
Variant(CHROM=chr22, POS=17553481, REF=T, ALT=C, ID=.)
Variant(CHROM=chr22, POS=17553617, REF=C, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17553617, REF=C, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17553617, REF=C, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17553701, REF=T, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17553701, REF=T, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17553701, REF=T, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17555125, REF=T, ALT=C, ID=.)
Variant(CHROM=chr22, POS=17555735, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=17556202, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=17557059, REF=C, ALT=T, ID=.)
Variant(CHROM=chr22, POS=17563719, REF=G, ALT=A, ID=.)
Variant(CH

2it [02:00, 74.34s/it]

...

Variant(CHROM=chr22, POS=50286302, REF=A, ALT=T, ID=.)
Variant(CHROM=chr22, POS=50286302, REF=A, ALT=T, ID=.)
Variant(CHROM=chr22, POS=50286302, REF=A, ALT=T, ID=.)
Variant(CHROM=chr22, POS=50287821, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50287821, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50289171, REF=C, ALT=T, ID=.)
Variant(CHROM=chr22, POS=50289171, REF=C, ALT=T, ID=.)
Variant(CHROM=chr22, POS=50289171, REF=C, ALT=T, ID=.)
Variant(CHROM=chr22, POS=50289171, REF=C, ALT=T, ID=.)
Variant(CHROM=chr22, POS=50289692, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=50289692, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=50289692, REF=A, ALT=G, ID=.)
Variant(CHROM=chr22, POS=50290590, REF=T, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50290590, REF=T, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50290590, REF=T, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50290590, REF=T, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50290610, REF=C, ALT=A, ID=.)
Variant(CHROM=chr22, POS=50290610, REF=C, ALT=A, ID=.)
Variant(CH

43it [28:01, 19.60s/it]

Variant(CHROM=chr22, POS=50574280, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50574280, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50574280, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50574280, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50575942, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=50575942, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=50575942, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=50575942, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=50575942, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=50575942, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=50575942, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=50575942, REF=G, ALT=A, ID=.)
Variant(CHROM=chr22, POS=50576554, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50576554, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50576554, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50576554, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50576554, REF=A, ALT=C, ID=.)
Variant(CHROM=chr22, POS=50576554, REF=A, ALT=C, ID=.)
Variant(CH

44it [28:15, 17.97s/it]

CPU times: user 3h 21min 10s, sys: 3h 26min 39s, total: 6h 47min 49s
Wall time: 28min 15s


In [17]:
out_preds = "/s/project/kipoi-cadd/data/models/MMSplice/sample_chr22_predictions.tsv"
predictions.to_csv(out_preds, sep="\t", index=False)
predictions.head()

,ID,exons,mmsplice_ref_acceptorIntron,mmsplice_ref_acceptor,mmsplice_ref_exon,mmsplice_ref_donor,mmsplice_ref_donorIntron,mmsplice_alt_acceptorIntron,mmsplice_alt_acceptor,mmsplice_alt_exon,mmsplice_alt_donor,mmsplice_alt_donorIntron
0,chr22:16639593:C:['G'],chr22_16639298_16640737:+,-4.037044,-4.128768,-3.991174,-3.331413,0.062515,-4.037044,-4.128768,-3.991174,-3.331413,0.062515
1,chr22:16639610:C:['T'],chr22_16639298_16640737:+,-4.037044,-4.128768,-3.991174,-3.331413,0.062515,-4.037044,-4.128768,-3.991174,-3.331413,0.062515
2,chr22:16639611:A:['G'],chr22_16639298_16640737:+,-4.037044,-4.128768,-3.991174,-3.331413,0.062515,-4.037044,-4.128768,-3.991174,-3.331413,0.062515
3,chr22:16639665:T:['A'],chr22_16639298_16640737:+,-4.037044,-4.128768,-3.991174,-3.331413,0.062515,-4.037044,-4.128768,-3.991174,-3.331413,0.062515
4,chr22:16639725:T:['G'],chr22_16639298_16640737:+,-4.037044,-4.128768,-3.991174,-3.331413,0.062515,-4.037044,-4.128768,-3.991174,-3.331413,0.062515


In [23]:
predictions.shape

(44826, 13)

In [18]:
# Summerize with maximum effect size
predictionsMax = max_varEff(predictions)

In [21]:
print(predictionsMax.shape)
out_preds_max = "/s/project/kipoi-cadd/data/models/MMSplice/sample_chr22_maxVarEff.tsv"
predictionsMax.to_csv(out_preds_max, sep="\t", index=False)
predictionsMax.head()

(17547, 13)


,ID,mmsplice_dlogitPsi,exons,mmsplice_ref_acceptorIntron,mmsplice_ref_acceptor,mmsplice_ref_exon,mmsplice_ref_donor,mmsplice_ref_donorIntron,mmsplice_alt_acceptorIntron,mmsplice_alt_acceptor,mmsplice_alt_exon,mmsplice_alt_donor,mmsplice_alt_donorIntron
0,chr22:16639593:C:['G'],0.000648,chr22_16639298_16640737:+,-4.037044,-4.128768,-3.991174,-3.331413,0.062515,-4.037044,-4.128768,-3.991174,-3.331413,0.062515
1,chr22:16639610:C:['T'],0.000648,chr22_16639298_16640737:+,-4.037044,-4.128768,-3.991174,-3.331413,0.062515,-4.037044,-4.128768,-3.991174,-3.331413,0.062515
2,chr22:16639611:A:['G'],0.000648,chr22_16639298_16640737:+,-4.037044,-4.128768,-3.991174,-3.331413,0.062515,-4.037044,-4.128768,-3.991174,-3.331413,0.062515
3,chr22:16639665:T:['A'],0.000648,chr22_16639298_16640737:+,-4.037044,-4.128768,-3.991174,-3.331413,0.062515,-4.037044,-4.128768,-3.991174,-3.331413,0.062515
4,chr22:16639725:T:['G'],0.000648,chr22_16639298_16640737:+,-4.037044,-4.128768,-3.991174,-3.331413,0.062515,-4.037044,-4.128768,-3.991174,-3.331413,0.062515


## Merge vcfs of SNVs

In [10]:
import pandas as pd

hum = "/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37/humanDerived_SNVs_chr22.vcf"
sim = "/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37/simulation_SNVs_chr22.vcf"
hum_df = pd.read_csv(hum, sep="\t", header=None)
hum_df_sample = hum_df.sample(65000, random_state=22)
sim_df = pd.read_csv(sim, sep="\t", header=None)
sim_df_sample = sim_df.sample(65000, random_state=22)
sim_df_sample.head()

,0,1,2,3,4
34229,22,23476778,.,G,A
64305,22,28809436,.,G,A
17862,22,20170134,.,G,A
24472,22,21556739,.,C,T
30774,22,22801542,.,A,G


In [11]:
hum_df_sample.to_csv(hum, sep="\t", header=None, index=False)
sim_df_sample.to_csv(sim, sep="\t", header=None, index=False)

In [12]:
from kipoi_cadd.utils import concatenate_vcf_files

directory = "/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37"
filenames = ["humanDerived_SNVs_chr22.vcf", "simulation_SNVs_chr22.vcf"]
output = "/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37/sample_chr22.vcf"
concatenate_vcf_files(directory, filenames, output)

/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37/humanDerived_SNVs_chr22.vcf
/s/project/kipoi-cadd/data/raw/v1.4/training_data/GRCh37/simulation_SNVs_chr22.vcf


In [51]:
! conda info --envs

# conda environments:
#
                         /opt/modules/i12g/anaconda/3-4.3.1/envs/py36
base                     /opt/modules/i12g/anaconda/3-5.0.1
DeepBind                 /opt/modules/i12g/anaconda/3-5.0.1/envs/DeepBind
DeepSEA                  /opt/modules/i12g/anaconda/3-5.0.1/envs/DeepSEA
FactorNet                /opt/modules/i12g/anaconda/3-5.0.1/envs/FactorNet
Python3.5Test            /opt/modules/i12g/anaconda/3-5.0.1/envs/Python3.5Test
RL                       /opt/modules/i12g/anaconda/3-5.0.1/envs/RL
avsec                    /opt/modules/i12g/anaconda/3-5.0.1/envs/avsec
cadd-env                 /opt/modules/i12g/anaconda/3-5.0.1/envs/cadd-env
cadd-env-py36            /opt/modules/i12g/anaconda/3-5.0.1/envs/cadd-env-py36
cadd13_py27              /opt/modules/i12g/anaconda/3-5.0.1/envs/cadd13_py27
cadd_temp                /opt/modules/i12g/anaconda/3-5.0.1/envs/cadd_temp
chipnexus                /opt/modules/i12g/anaconda/3-5.0.1/envs/chipnexus
deep-rbp                 /